In [12]:
with open('text.txt','r',encoding='utf-8') as file:
    full_text = file.read()

In [13]:
import yake
kw_extractor = yake.KeywordExtractor(top=10, stopwords=None)
keywords = kw_extractor.extract_keywords(full_text)
for kw, v in keywords:
  print("Keyphrase: ",kw, ": score", v)

Keyphrase:  octet-stream type posts : score 0.012957445239480963
Keyphrase:  form-data type forms : score 0.016356499078172874
Keyphrase:  octet-stream type : score 0.028378849701799807
Keyphrase:  form-data type : score 0.028378849701799807
Keyphrase:  working using application : score 0.028785995937607795
Keyphrase:  images using flask : score 0.028785995937607795
Keyphrase:  upload images : score 0.06424318756146702
Keyphrase:  request working : score 0.07076208983559576
Keyphrase:  type posts : score 0.08189860827394943
Keyphrase:  type : score 0.09377536030868192


In [14]:
from multi_rake import Rake
rake = Rake()
keywords = rake.apply(full_text)
print(keywords[:10])

[('application/octet-stream type posts', 9.0), ('multipart/form-data type forms', 9.0), ('post based route', 7.0), ('request working', 4.0), ('upload images', 4.0), ('request element', 4.0), ('ajax call', 4.0), ('standard form', 3.5), ('post', 2.0), ('route', 2.0)]


In [15]:
from summa import keywords
TR_keywords = keywords.keywords(full_text, scores=True)
print(TR_keywords[0:10])

[('worked leveraging', 0.53008566495395), ('request working', 0.5300856649539487), ('werkzeug', 0.2650428324769752)]


In [19]:
from keybert import KeyBERT
kw_model = KeyBERT(model='all-mpnet-base-v2')
keywords = kw_model.extract_keywords(full_text, 

                                     keyphrase_ngram_range=(1, 3), 

                                     stop_words='english',

                                     highlight=False,

                                     top_n=10)

keywords_list= list(dict(keywords).keys())
print(keywords_list)

['images using flask', 'support multiple upload', 'upload files', 'upload images using', 'multiple upload files', 'multiple upload', 'upload images', 'multipart form data', 'multipart form', 'using flask extended']


In [17]:
import pke
extractor = pke.unsupervised.TopicRank()
extractor.load_document(input='text.txt', language='en')
extractor.candidate_selection()
extractor.candidate_weighting()
keyphrases = extractor.get_n_best(n=10)

['images using flask', 'support multiple upload']
